# Composite Trapezoidal Rule: Exercises h, i, j

This notebook implements and studies the composite trapezoidal rule for the tasks:
- **h** Implement the composite trapezoidal rule
- **i** Use the method for $\exp$ on [0,1], compute the actual error, and find the minimal subintervals for a given tolerance
- **j** Estimate the numerical convergence rate for $\exp$ and $\sqrt{x}$


## Parameters
- `A, B`: integration interval
- `N_FROM_G`: the n referenced from part g, used in part i
- `TOL`: error tolerance for part i
- `ORDER_BASE_NS`: base n values used to estimate observed order p in part j


In [13]:
import numpy as np


## h) Composite trapezoidal rule
We implement a reusable function `composite_trapezoid(f, a, b, n)` that returns the approximation of $\int_a^b f(x)\,dx$ using **n** uniform subintervals.

In [14]:
from typing import Callable

def composite_trapezoid(f: Callable[[np.ndarray], np.ndarray], a: float, b: float, n: int) -> float:
    """Composite trapezoidal rule on [a, b] with n subintervals.

    Parameters
    ----------
    f : callable mapping array -> array
        Vectorized integrand.
    a, b : float
        Integration limits with a < b.
    n : int
        Number of subintervals, n >= 1.

    Returns
    -------
    float
        Trapezoidal approximation of the integral.
    """
    if n < 1:
        raise ValueError("n must be >= 1")
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return (h / 2.0) * (fx[0] + 2.0 * fx[1:-1].sum() + fx[-1])


### Helper utilities
We provide small helpers for exact integrals on [0,1], error computation, and order estimation.

In [15]:
def exact_integral_exp_on_01() -> float:
    return np.e - 1.0

def exact_integral_sqrt_on_01() -> float:
    return 2.0 / 3.0

def trap_error(f, a: float, b: float, n: int, exact_value: float) -> float:
    """Absolute error for trapezoid with n subintervals."""
    approx = composite_trapezoid(f, a, b, n)
    return abs(approx - exact_value)

def estimate_order_via_triplet(f, a: float, b: float, n: int, exact_value: float) -> float:
    """Return observed order p using errors at n, 2n, 4n:
    p = log(E_{2n}/E_{4n}) / log(E_n/E_{2n})."""
    E1 = trap_error(f, a, b, n, exact_value)
    E2 = trap_error(f, a, b, 2*n, exact_value)
    E3 = trap_error(f, a, b, 4*n, exact_value)
    if E1 == 0 or E2 == 0 or E3 == 0:
        return float('nan')
    import numpy as _np
    return float(_np.log(E2/E3) / _np.log(E1/E2))

def minimal_n_for_tolerance(f, a: float, b: float, exact_value: float, tol: float, n0: int = 1) -> int:
    """Find the smallest n such that error <= tol by doubling, then shrinking.
    Returns the minimal n.
    """
    # Grow n until we beat tolerance
    n = max(1, n0)
    while trap_error(f, a, b, n, exact_value) > tol:
        n *= 2

    # Binary search in (n/2, n]
    lo = max(1, n // 2)
    hi = n
    while lo + 1 < hi:
        mid = (lo + hi) // 2
        if trap_error(f, a, b, mid, exact_value) <= tol:
            hi = mid
        else:
            lo = mid
    return hi


In [16]:
# Global parameters for exercises on [0,1]
A, B = 0.0, 1.0
N_FROM_G = 16        # taken from the earlier part g in your workflow
TOL = 1e-5           # target error bound in part i
ORDER_BASE_NS = [4, 8, 16, 32, 64, 128]  # base n for observed order estimates


## i) Apply to exp on [0,1], compute error, and find minimal n for tolerance
We evaluate the trapezoidal rule for `n = N_FROM_G`, compute the absolute error, and determine the minimal subinterval count such that the error is at most `TOL`.

In [17]:
def f_exp(x: np.ndarray) -> np.ndarray:
    return np.exp(x)

exact_exp = exact_integral_exp_on_01()
approx_g = composite_trapezoid(f_exp, A, B, N_FROM_G)
error_g = abs(approx_g - exact_exp)

n_min = minimal_n_for_tolerance(f_exp, A, B, exact_exp, TOL, n0=2)

print(f"Approximation for exp with n = {N_FROM_G}: {approx_g:.16f}")
print(f"Actual error for n = {N_FROM_G}: {error_g:.3e}")
print(f"Minimal n for tolerance {TOL:g}: {n_min}")


Approximation for exp with n = 16: 1.7188411285799945
Actual error for n = 16: 5.593e-04
Minimal n for tolerance 1e-05: 120


## j) Estimate numerical convergence rates for exp and sqrt
Assume errors satisfy $E^{n}_{[0,1]}[f] \approx C h^p$ with $h=(b-a)/n$.
We estimate p from three successive refinements n, 2n, 4n and then average over several base n values.

In [18]:
def f_sqrt(x: np.ndarray) -> np.ndarray:
    return np.sqrt(x)

exact_sqrt = exact_integral_sqrt_on_01()

def observed_p_over_ns(f, exact_value):
    ps = []
    for n in ORDER_BASE_NS:
        p = estimate_order_via_triplet(f, A, B, n, exact_value)
        if np.isfinite(p):
            ps.append(p)
    return np.array(ps)

p_exp = observed_p_over_ns(f_exp, exact_exp)
p_sqrt = observed_p_over_ns(f_sqrt, exact_sqrt)

print("Observed p for exp over base n:")
print(p_exp)
print(f"p_exp mean = {p_exp.mean():.6f}, std = {p_exp.std(ddof=1):.6f}")

print("\nObserved p for sqrt over base n:")
print(p_sqrt)
print(f"p_sqrt mean = {p_sqrt.mean():.6f}, std = {p_sqrt.std(ddof=1):.6f}")


Observed p for exp over base n:
[1.00042226 1.00010564 1.00002641 1.0000066  1.00000165 1.00000041]
p_exp mean = 1.000094, std = 0.000166

Observed p for sqrt over base n:
[1.00987364 1.00663297 1.00451888 1.00311226 1.00216032 1.00150783]
p_sqrt mean = 1.004634, std = 0.003152
